# L2d: Debug our Fibonacci Calculation
In this lab, we'll continue the development of our Fibonacci sequence calculations. Previously, we constructed an empty `MyFibonacciSequenceModel` struct instance, by calling the default `MyFibonacciSequenceModel()` constructor method. 

> __Issue:__ In the lecture implementation, [the mutable `MyFibonacciSequenceModel` struct](src/Types.jl) instances were not properly initialized with the required fields. If a user was unaware of the implementation details, they could try to use this problem object and encounter errors.

In this lab, we'll analyze and improve [a `build(...)` method](src/Factory.jl) which is responsible for properly constructing [`MyFibonacciSequenceModel` instances](src/Types.jl), where our build process should be robust to invalid inputs and ensure that the model is correctly initialized. We'll then pass this problem object to [the `fibonacci!(...)` method](src/Compute.jl) to see what happens.

However, before we get going, break up into groups, and take __10 minutes__ to review the lab and its associated Julia files. 
___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include__: The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. 

Let's set up the computational environment.

In [1]:
include(joinpath(@__DIR__, "Include-student.jl")); # what is this doing?

For additional information on Julia functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). In addition, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on its functions, types, and data. 
___

## Task 1: Create a build method for MyFibonacciSequenceModel
In this task, we'll work with [a `build(...)` method](src/Factory.jl) in the [Factory.jl file](src/Factory.jl) that constructs a `MyFibonacciSequenceModel` instance with a specified size and default values for the other fields.

This method will ensure that the model is properly initialized __before__ being passed [to the `fibonacci!(...)` method](src/Compute.jl).

__Requirements__:
* __Arguments__: The [`build(...)`](src/Factory.jl) method will take the type of thing we want to build, i.e., [`MyFibonacciSequenceModel`](src/Types.jl), the sequence size `n::Int` and the default value `defaultvalue::Int` parameters. These data will be passed into the build method as a `data::NamedTuple` instance, see [the documentation on NamedTuples here](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple). The `build(...)` method returns a properly constructed `MyFibonacciSequenceModel` instance.
* __Error conditions__: If the required fields are missing or invalid, the method should return an error message and a `Nothing` value for the model being constructed.

Break up into groups, and take __5 minutes__ to analyze [the `build(...)` method](src/Factory.jl).
___

## Task 2: Let's test our build method
In this task, we will test [the `build(...)` method](src/Factory.jl) to ensure it correctly constructs a `MyFibonacciSequenceModel` instance with the specified size and default values.

> __Test cases.__ We'll consider several different test cases, where different values of `n` and `defaultvalue` are used to create the model instance. Sometimes these values will be valid, and sometimes they will not. However, [our `build(...)` method](src/Factory.jl) should handle these cases gracefully, returning a valid model instance or throwing an appropriate error.

### Happy Path
Let's start with the first case, the so called _happy path_, where we provide valid `n::Int64` and `defaultvalue::Int64` arguments to [the `build(...)` method](src/Factory.jl). This should return a properly constructed `MyFibonacciSequenceModel` instance:

In [2]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=10, defaultvalue=0)); # build a new model

This seems to have run! But let's do a few checks to make sure we are doing what we think we are doing. 

* _Is the `my_sequence_model` instance the correct type?_ Let's check this using [the `isa(...)` method](https://docs.julialang.org/en/v1/base/base/#Core.isa) in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to ensure that the `my_sequence_model` instance is of type [`MyFibonacciSequenceModel`](src/Types.jl).
* _Is the sequence dictionary initialized correctly?_ The `sequence::Dict{Int64,Int64}` dictionary should have a length equal to `n`, and all elements should be equal to the `defaultvalue`. We can use the [length(...) method](https://docs.julialang.org/en/v1/base/collections/#Base.length) to check the length of the `sequence` field, and [the all(...) method](https://docs.julialang.org/en/v1/base/collections/#Base.all-Tuple%7BAny%7D) to check that all elements are equal to the `defaultvalue`.

If any of these checks fail, [an `AssertionError`](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown with a descriptive message.

In [ ]:
let

    # Check 1: Check the model type -
    @assert my_sequence_model isa MyFibonacciSequenceModel "Error: The model is not of type MyFibonacciSequenceModel";

    # Check 2: Check the sequence length and default values -
    @assert length(my_sequence_model.sequence) == 10 "Error: The sequence length is not equal to 10";
    @assert all(k -> my_sequence_model.sequence[k] == 0, keys(my_sequence_model.sequence)) "Error: Not all sequence values are equal to 0";
end

### Error Handling
If all our checks pass, then we know that when provided with the correct parameters, the model is built correctly. However, sometimes users make mistakes, they can't read our mind (or we did a bad job of documenting the code). So, let's also test some error cases where users enter invalid parameters.

> __Current condition:__ Right now (with all the debugging checks disabled), [the `build(...)` method](src/Factory.jl) will blow up if we did not provide a valid `n::Int64` or `defaultvalue::Int64` parameter. 

What happens if we do not provide a `defaultvalue` parameter in `data::NamedTuple`?

In [4]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=10,)); # build a new model|

┌ Error: Ooops! Missing required field: defaultvalue. Cannot build the model, returning nothing.
└ @ Main /Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-2/L2d/solution/Factory.jl:22


Without the `defaultvalue` parameter, the `build(...)` method will throw the error: __type NamedTuple has no field defaultvalue__ Let's handle this gracefully by ensuring that all parameters are provided. 

> `Uncomment` the first error handling block in the `build(...)` method, restart the Julia Kernel, and run all cells. The error checking logic checks that the required fields are provided in the `data::NamedTuple` instance. If a field is missing, an `ArgumentError` with a descriptive message is thrown.

Rerun the [updated `build(...)` method](src/Factory.jl) with a `data::NamedTuple` instance that does not contain the `defaultvalue::Int64` field.  What happens now?

In [5]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=10,)); # build a new model

┌ Error: Ooops! Missing required field: defaultvalue. Cannot build the model, returning nothing.
└ @ Main /Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-2/L2d/solution/Factory.jl:22


Confirm that the model is not built, and the `my_sequence_model` instance is [`nothing`](https://docs.julialang.org/en/v1/base/constants/#Core.nothing) as expected. We can use the [`isnothing(...)`](https://docs.julialang.org/en/v1/base/base/#Base.isnothing) method in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if `my_sequence_model` is `nothing`.

In [ ]:
@assert isnothing(my_sequence_model) == true "Error: The model should be nothing, but it contains data";

Next, let's check what happens if we provide an invalid `n` parameter. For example, if we provide a negative value for `n`, the `build(...)` method should throw an `ArgumentError` with a descriptive message. 

> __Current:__ Without the next block of error checking logic, [the `build(...)` method](src/Factory.jl) returns an empty `MyFibonacciSequenceModel` instance when passed bad values for the `n` parameter (or the `defaultvalue` parameter), which is not what we want (this is a strange state for the model to be in).

> `Uncomment` the second and third error handling blocks in [the `build(...)` method](src/Factory.jl), restart the Julia Kernel and run all cells. 

> __What should happen?__ The error checking logic checks that the `n::Int64` parameter is a positive integer, and that the `defaultvalue::Int64` parameter is an integer. If either of these conditions is not met, a `warning` message is shown.

So what do we see?

In [7]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=-10, defaultvalue = -1)); # build a new model

┌ Warning: Ooops! Invalid value for n: -10. Using default value: 10.
└ @ Main /Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-2/L2d/solution/Factory.jl:38


While this works a little better, we still have a problem. 

> __Problem:__ The [`build(...)` method](src/Factory.jl) returns an empty [`MyFibonacciSequenceModel` instance](src/Types.jl) when passed bad values for the `n::Int64` parameter, a default value for `n` is set on the model, but the `sequence::Dict{Int64,Int64}` dictionary is empty, which is not what we want (this is a strange state for the model to be in).

What is stored in the `sequence::Dict{Int64,Int64}` dictionary?

In [8]:
my_sequence_model.sequence

Dict{Int64, Int64} with 10 entries:
  0 => -1
  4 => -1
  5 => -1
  6 => -1
  2 => -1
  7 => -1
  9 => -1
  8 => -1
  3 => -1
  1 => -1

`Uncomment` the last error handling block in [the `build(...)` method](src/Factory.jl), restart the Julia Kernel and run all cells. The final error checking block fills the empty `sequence::Dict{Int64,Int64}` dictionary with the `defaultvalue::Int64` for all keys from `0` to `n-1`. 

In [9]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=-10, defaultvalue = -1)); # build a new model

┌ Warning: Ooops! Invalid value for n: -10. Using default value: 10.
└ @ Main /Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-2/L2d/solution/Factory.jl:38


We received a `warning` message, but a warning is not an error, so the `my_sequence_model` instance is still created, and we can use it to compute the Fibonacci numbers! Let's check the instance that gets returned [by the `build(...)` method](src/Factory.jl).
> __Expectation__: We expect the `my_sequence_model` instance to not be `nothing`, the model will be [type `MyFibonacciSequenceModel`](src/Types.jl), and the `sequence::Dict{Int64,Int64}` dictionary should be populated with the default value for all keys from `0` to `n-1`.

Let's check these four conditions using [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert):

In [ ]:
let

    # initialize -
    correct_sequence_length = 10; # this is the default value for n
    correct_default_value = -1; # this is the default value for the defaultvalue parameter

    # Check 1: Check the model type -
    @assert isnothing(my_sequence_model) == false "Error: The model is nothing, it should be a MyFibonacciSequenceModel";
    @assert my_sequence_model isa MyFibonacciSequenceModel "Error: The model is not of type MyFibonacciSequenceModel";

    # Check 2: Check the sequence length and default values -
    @assert length(my_sequence_model.sequence) == correct_sequence_length "Error: The sequence length is not equal to 10";
    @assert all(k -> my_sequence_model.sequence[k] == correct_default_value, keys(my_sequence_model.sequence)) "Error: Not all sequence values are equal to -1";
end

### Compute the sequence
If all of our error handling is in place, we should be able to compute the Fibonacci sequence without any issues. Let's test this by calling [the `fibonacci!(...)` method](src/Compute.jl) on our `my_sequence_model::MyFibonacciSequenceModel` instance.

You can play around with different values for `n` and `defaultvalue` to see how the model behaves.

In [13]:
result, my_test_sequence_model = let

    # initialize -
    my_test_sequence_model = build(MyFibonacciSequenceModel, (
        n = 10, # length of the sequence
        defaultvalue = 0 # default value for the sequence elements
    )); # build a new model

    @show my_test_sequence_model

    result = fibonacci!(my_test_sequence_model, 
        iterationmodel = MyForLoopIterationModel()); # compute the Fibonacci sequence

    # return -
    result, my_test_sequence_model
end

my_test_sequence_model = MyFibonacciSequenceModel(10, Dict(0 => 0, 4 => 0, 5 => 0, 6 => 0, 2 => 0, 7 => 0, 9 => 0, 8 => 0, 3 => 0, 1 => 0))


┌ Info: Debug message: We are using the for loop iteration model
└ @ Main /Users/jdv27/Desktop/julia_work/CHEME-5800-Instances/Fall-2025/CHEME-5800-Labs-Fall-2025/labs/week-2/L2d/solution/Compute.jl:4


(Dict(5 => 5, 8 => 21, 1 => 1, 0 => 0, 6 => 8, 9 => 34, 3 => 2, 7 => 13, 4 => 3, 2 => 1…), MyFibonacciSequenceModel(10, Dict(5 => 5, 8 => 21, 1 => 1, 0 => 0, 6 => 8, 9 => 34, 3 => 2, 7 => 13, 4 => 3, 2 => 1…)))

## Summary

__Congratulations!__ You have successfully completed the error handling and debugging lab. In this activity, you learned several important concepts:

### Key Learning Outcomes:
1. **Robust Factory Methods**: How to implement factory methods with comprehensive error checking that validate input parameters before object construction
2. **Error vs. Warning Handling**: The difference between throwing errors (which stop execution) and issuing warnings (which allow execution to continue with default values)
3. **Assertion Testing**: How to use the `@assert` macro to validate object state and ensure your code behaves as expected
4. **Graceful Error Recovery**: Best practices for handling invalid inputs by returning meaningful error messages and `nothing` values rather than allowing the program to crash
5. **Progressive Error Handling**: How to implement layered error checking that catches different types of problems at different stages

### Best Practices Demonstrated:
- Always validate input parameters in factory/constructor methods
- Use descriptive error messages that help users understand what went wrong
- Test both "happy path" scenarios and error conditions
- Use assertions to validate that objects are in expected states
- Design error handling that provides useful feedback rather than cryptic failures

These skills will be essential as you develop more complex Julia applications where robust error handling can make the difference between a program that crashes unexpectedly and one that provides helpful feedback to users when things go wrong.
___